#  <center> Problem Set 6 <center>

<center> 3.C01/3.C51, 10.C01/10.C51 <center>

In [1]:
# Cheminformatics
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors,Crippen
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

# Arrays
import numpy as np
import pandas as pd

# Scikit-learn
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPRegressor

# Plotting 
import matplotlib.pyplot as plt
import matplotlib as mpl

# Personal utility package
import package.plot
from package.plot import get_size_inches

from pathlib import Path

# Machine learning
import torch
from lightning import pytorch as pl
from lightning.pytorch.loggers import CSVLogger

# 10.1021/acs.jcim.3c01250
from chemprop import data, featurizers, models, nn

# Utility
from datetime import datetime
from dataclasses import dataclass

## Part 1: Baseline Regression Methods

### Part 1.1: (5 points) Prepare Dataset

In [2]:
data_dir = Path.cwd() / "data"
train_file = data_dir / "solvation_train.csv"
test_file = data_dir / "solvation_test.csv"
prop_file = data_dir / "molecule_props.csv"
df = pd.read_csv(train_file) # load data
mol_prop = pd.read_csv(prop_file)

Generate fingerprints (e.g. a Morgan fingerprint).

In [5]:
smiles = 'CN1C=NC2=C1C(=O)N(C(=O)N2C)C'
mol = Chem.MolFromSmiles(smiles) # load SMILES into RDKit
fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=512) # morgan fingerprint 
fp_array = np.zeros((1,), int) # convert to numpy array
DataStructs.ConvertToNumpyArray(fp, fp_array)

Generate various chemical properties.

In [6]:
MolWt = Descriptors.ExactMolWt(mol) # molecular weight
TPSA = Chem.rdMolDescriptors.CalcTPSA(mol) # Topological Polar Surface Area
nRotB = Descriptors.NumRotatableBonds(mol) # number of rotable bonds
HBD = Descriptors.NumHDonors(mol) # number of H bond donors
HBA = Descriptors.NumHAcceptors(mol) # number of H bond acceptors
logP = Descriptors.MolLogP(mol) # LogP
dct_pol = dict(zip(mol_prop["SMILES"],mol_prop["polarizability"]))
dct_dip = dict(zip(mol_prop["SMILES"],mol_prop["dipole"]))

Create a feature set with concatenated physical descriptors.

In [8]:
def featurize(smiles):
    mol = Chem.MolFromSmiles(smiles) # load SMILES into RDKit
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=512) # morgan fingerprint 
    fp_array = np.zeros((1,), int) # convert to numpy array
    DataStructs.ConvertToNumpyArray(fp, fp_array)
    
    MolWt = Descriptors.ExactMolWt(mol) # molecular weight
    TPSA = Chem.rdMolDescriptors.CalcTPSA(mol) # Topological Polar Surface Area
    nRotB = Descriptors.NumRotatableBonds(mol) # number of rotable bonds
    HBD = Descriptors.NumHDonors(mol) # number of H bond donors
    HBA = Descriptors.NumHAcceptors(mol) # number of H bond acceptors
    logP = Descriptors.MolLogP(mol) # LogP
    # polar = dct_pol[smiles]
    # dipole = dct_dip[smiles]

    return np.hstack([fp_array, [MolWt, TPSA, nRotB, HBD, HBA, logP]])

In [9]:
y = df["logK"]
X_solvent = np.stack(df["Solvent"].apply(featurize).values)
X_solute = np.stack(df["Solute"].apply(featurize).values)
X = np.hstack([X_solvent,X_solute])

### 1.2 (10 points) Linear Regression

Train a linear regression model and report a 5-fold cross-validated R^2.

In [10]:
scaler = StandardScaler()
scores = cross_val_score(LinearRegression(), X, y, cv = 5, scoring = "r2")
print(f"No Regularization R2: {scores.mean()}")
scores = cross_val_score(Lasso(), X, y, cv = 5, scoring = "r2")
print(f"Lasso R2: {scores.mean()}")
scores = cross_val_score(Ridge(), X, y, cv = 5, scoring = "r2")
print(f"Ridge R2: {scores.mean()}")

No Regularization R2: -1.0751368220270634e+16
Lasso R2: 0.4794894887076525
Ridge R2: 0.9224699304617175


### 1.3 (10 points) MLP Regression

Train an MLP regression model and report a 5-fold cross-validated R^2.

In [11]:
# initialize a MLP with the specified hyperparameters
kwargs = {"hidden_layer_sizes" : (256,256,256),
          "activation" : "relu",
          "alpha" : 0.16,
          "solver" : "adam",
          "early_stopping" : False}
mlp =  MLPRegressor(**kwargs)
pipe = Pipeline([('scaler', StandardScaler()), ('model', mlp)])
scores = cross_val_score(pipe, X, y, cv = 5, scoring = "r2")
print(f"MLP R2: {scores.mean()}")

MLP R2: 0.9643595568587566


/home/gridsan/ddavid/.conda/envs/torch/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


## Part 2: (50 points) Machine Learning Competition and Report

You can start a new notebook here to put all your models.

In [292]:
def save_submission(prediction, filename):
    '''
    Utility function to dump a submission file.

    prediction (numpy.array): 1d numpy array contains your prediction
    filename (str): file path to where you want to save the result
    '''
    sub = pd.DataFrame( {'index': list(range(len(prediction))), 'logK': prediction } )
    sub.to_csv(filename, index=False)

In [293]:
@dataclass
class PropFeaturizer(featurizers.Featurizer):
    def __init__(self, features: list):
        self.features = features

    def __len__(self) -> int:
        return len(self.features)

    def MolWt(self, mol):
        return Descriptors.ExactMolWt(mol)

    def TPSA(self, mol):
        return Chem.rdMolDescriptors.CalcTPSA(mol)
    
    def LASA(self, mol):
        return Chem.rdMolDescriptors.CalcLabuteASA(mol)

    def NumRotatableBonds(self, mol):
        return Descriptors.NumRotatableBonds(mol)

    def NumHDonors(self, mol):
        return Descriptors.NumHDonors(mol)

    def NumHAcceptors(self, mol):
        return Descriptors.NumHAcceptors(mol)

    def MolLogP(self, mol):
        return Descriptors.MolLogP(mol)

    def MolMR(self, mol):
        return Descriptors.MolMR(mol)

    def AromProp(self, mol):
        return len(list(mol.GetAromaticAtoms())) / mol.GetNumHeavyAtoms()

    def __call__(self, mol: Chem.Mol) -> np.ndarray:
        return np.array([getattr(self,feature)(mol) for feature in self.features])

In [294]:
from chemprop.data.collate import NamedTuple, BatchMolGraph, Tensor, Iterable, Datum, collate_batch
from chemprop.data.dataloader import MoleculeDataset, ReactionDataset, MulticomponentDataset, DataLoader
import warnings

class MulticomponentTrainingBatch(NamedTuple):
    bmgs: list[BatchMolGraph]
    V_ds: list[Tensor | None]
    X_d: Tensor | None
    Y: Tensor | None
    w: Tensor
    lt_mask: Tensor | None
    gt_mask: Tensor | None


def custom_collate_multicomponent(batches: Iterable[Iterable[Datum]]) -> MulticomponentTrainingBatch:
    tbs = [collate_batch(batch) for batch in zip(*batches)]
    return MulticomponentTrainingBatch(
        [tb.bmg for tb in tbs],
        [tb.V_d for tb in tbs],
        torch.cat([tbs[0].X_d,tbs[1].X_d],axis=1),
        tbs[0].Y,
        tbs[0].w,
        tbs[0].lt_mask,
        tbs[0].gt_mask,
    )

def custom_build_dataloader(
    dataset: MoleculeDataset | ReactionDataset | MulticomponentDataset,
    batch_size: int = 64,
    num_workers: int = 0,
    class_balance: bool = False,
    seed: int | None = None,
    shuffle: bool = True,
    **kwargs,
):

    if class_balance:
        sampler = ClassBalanceSampler(dataset.Y, seed, shuffle)
    elif shuffle and seed is not None:
        sampler = SeededSampler(len(dataset), seed)
    else:
        sampler = None

    if isinstance(dataset, MulticomponentDataset):
        collate_fn = custom_collate_multicomponent
    else:
        collate_fn = collate_batch

    if len(dataset) % batch_size == 1:
        warnings.warn(
            f"Dropping last batch of size 1 to avoid issues with batch normalization \
(dataset size = {len(dataset)}, batch_size = {batch_size})"
        )
        drop_last = True
    else:
        drop_last = False

    return DataLoader(
        dataset,
        batch_size,
        sampler is None and shuffle,
        sampler,
        num_workers=num_workers,
        collate_fn=collate_fn,
        drop_last=drop_last,
        **kwargs,
    )

In [295]:
features = ["TPSA", "LASA", "NumRotatableBonds", "NumHDonors", "NumHAcceptors", "MolLogP", "MolMR", "AromProp"]

def get_loaders(features, split_type="random", split = (0.8, 0.1, 0.1)):
    data_dir = Path.cwd() / "data"
    train_file = data_dir / "solvation_train.csv"
    test_file = data_dir / "solvation_test.csv"
    prop_file = data_dir / "molecule_props.csv"
    smiles_columns = ['Solute', 'Solvent'] # name of the column containing SMILES strings
    target_columns = ['logK'] # list of names of the columns containing targets
    df_input = pd.read_csv(train_file)
    smiss = df_input.loc[:, smiles_columns].values
    ys = df_input.loc[:, target_columns].values

    mfs = [PropFeaturizer(features)]
    all_data = [[data.MoleculeDatapoint.from_smi(smis[0], y, mfs=mfs) for smis, y in zip(smiss, ys)]]
    all_data += [[data.MoleculeDatapoint.from_smi(smis[i], mfs=mfs) for smis in smiss] for i in range(1, len(smiles_columns))]

    component_to_split_by = 0
    mols = [d.mol for d in all_data[component_to_split_by]]

    train_indices, val_indices, test_indices = data.make_split_indices(mols, split_type, split)
    
    train_data, val_data, test_data = data.split_data_by_indices(
        all_data, train_indices, val_indices, test_indices
    )

    featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

    train_datasets = [data.MoleculeDataset(train_data[i], featurizer) for i in range(len(smiles_columns))]
    train_mcdset = data.MulticomponentDataset(train_datasets)
    scaler = train_mcdset.normalize_targets()
    train_loader = custom_build_dataloader(train_mcdset)

    val_datasets = [data.MoleculeDataset(val_data[i], featurizer) for i in range(len(smiles_columns))]
    val_mcdset = data.MulticomponentDataset(val_datasets)
    val_mcdset.normalize_targets(scaler)
    val_loader = custom_build_dataloader(val_mcdset, shuffle=False)

    test_datasets = [data.MoleculeDataset(test_data[i], featurizer) for i in range(len(smiles_columns))]
    test_mcdset = data.MulticomponentDataset(test_datasets)
    test_loader = custom_build_dataloader(test_mcdset, shuffle=False)
    
    return train_loader, val_loader, test_loader

In [299]:
train_loader, val_loader, test_loader = get_loaders(features, split = (0.99, 0.01, 0))

/tmp/ipykernel_2407107/1504296857.py:50: UserWarning: Dropping last batch of size 1 to avoid issues with batch normalization (dataset size = 1, batch_size = 64)
  warnings.warn(


In [300]:
mcmp = nn.MulticomponentMessagePassing(
    blocks=[nn.BondMessagePassing(depth=6) for _ in range(len(smiles_columns))],
    n_components=len(smiles_columns),
)

agg = nn.MeanAggregation()

output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
ffn = nn.RegressionFFN(
    input_dim=mcmp.output_dim + (len(smiles_columns)) * np.sum([i.__len__() for i in mfs]),
    output_transform=output_transform,
    hidden_dim=1900,
    n_layers=2,
    dropout=0.008,
)

metric_list = [nn.metrics.RMSEMetric(), nn.metrics.MAEMetric(), nn.metrics.R2Metric()] # Only the first metric is used for training and early stopping

mcmpnn = models.multi.MulticomponentMPNN(
    mcmp,
    agg,
    ffn,
    metrics=metric_list,
)

/home/gridsan/ddavid/.conda/envs/torch/lib/python3.12/site-packages/lightning/pytorch/utilities/parsing.py:199: Attribute 'output_transform' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['output_transform'])`.


In [301]:
logger = CSVLogger("logs", name="model")

trainer = pl.Trainer(
    logger=logger,
    enable_checkpointing=True,
    enable_progress_bar=True,
    accelerator="gpu",
    devices=1,
    max_epochs=200, # number of epochs to train for
)

trainer.fit(mcmpnn, train_loader, val_loader)

/home/gridsan/ddavid/.conda/envs/torch/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/gridsan/ddavid/.conda/envs/torch/lib/python3.1 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.


/home/gridsan/ddavid/.conda/envs/torch/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=79` in the `DataLoader` to improve performance.

  | Name            | Type                         | Params
-----------------------------------------------------------------
0 | message_passing | MulticomponentMessagePassing | 455 K 
1 | agg             | MeanAggregation              | 0     
2 | bn              | BatchNorm1d                  | 1.2 K 
3 | predictor       | RegressionFFN                | 4.8 M 
4 | X_d_transform   | Identity                     | 0     
-----------------------------------------------------------------
5.2 M     Trainable params
0         Non-trainable params
5.2 M     Total params
20.971    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/gridsan/ddavid/.conda/envs/torch/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=79` in the `DataLoader` to improve performance.


Epoch 103: 100%|██████████| 57/57 [00:01<00:00, 37.35it/s, v_num=2, train_loss=0.0035, val_loss=0.0666] 

In [ ]:
results = trainer.test(mcmpnn, test_loader)

/home/gridsan/ddavid/.conda/envs/torch/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/gridsan/ddavid/.conda/envs/torch/lib/python3.1 ...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/gridsan/ddavid/.conda/envs/torch/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=79` in the `DataLoader` to improve performance.


Testing DataLoader 0:   0%|          | 0/6 [00:00<?, ?it/s]

/home/gridsan/ddavid/.conda/envs/torch/lib/python3.12/site-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [ ]:
kaggle_path = data_dir / "solvation_test.csv"
df_kaggle = pd.read_csv(kaggle_path)
smiss_kaggle = df_kaggle.loc[:, smiles_columns].values
kaggle_data = [[data.MoleculeDatapoint.from_smi(smis[i], mfs=mfs) for smis in smiss_kaggle] for i in range(len(smiles_columns))]
kaggle_datasets = [data.MoleculeDataset(kaggle_data[i], featurizer) for i in range(len(smiles_columns))]
kaggle_mcdset = data.MulticomponentDataset(kaggle_datasets)
kaggle_loader = custom_build_dataloader(kaggle_mcdset,shuffle=False)

In [ ]:
test_preds = trainer.predict(mcmpnn, kaggle_loader)
test_preds = np.concatenate(test_preds, axis=0)

/home/gridsan/ddavid/.conda/envs/torch/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/gridsan/ddavid/.conda/envs/torch/lib/python3.1 ...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/gridsan/ddavid/.conda/envs/torch/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=79` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 25/25 [00:00<00:00, 46.19it/s]


,index,Solvent,Solute,pred
0,0,CCCCCO,[Ne],-1.531519
1,1,CC(C)CC(C)(C)C,CCCCO,2.774545
2,2,c1ccc(cc1)[N+](=O)[O-],Cc1ccccc1,3.344532
3,3,C/C(=N/C)/O,CC(C)O,3.661321
4,4,CCOC(=O)C,CCCCCC,2.459938
...,...,...,...,...
1564,1564,CCCCCC,c1ccc(c(c1)Cl)Cl,4.454704
1565,1565,CC(C)(C)O,C(=O)=O,0.411111
1566,1566,CCCOCC,CO,2.276852
1567,1567,CCCCCCC,CCCC=C,2.288224


In [ ]:
formatted_time = datetime.now().strftime("%Y%m%d-%H%M")
save_submission(test_preds,f"kaggle/chemprop_{formatted_time}.csv")